In [2]:
import catboost as cb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [3]:
my_xtrain = pd.read_csv("../ourfeatures/updated_xfeature.csv")
my_ytrain = pd.read_csv("../ourfeatures/my_big_featurey.csv")

In [4]:
my_xtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377346 entries, 0 to 377345
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   transaction_number_max       377346 non-null  int64  
 1   amnt_min                     377346 non-null  float64
 2   amnt_max                     377346 non-null  float64
 3   amnt_mean                    377346 non-null  float64
 4   amnt_median                  377346 non-null  float64
 5   currency_num_unique_values   377346 non-null  int64  
 6   operation_type_group_median  377346 non-null  float64
 7   operation_kind_median        377346 non-null  float64
 8   income_flag_median           377346 non-null  float64
 9   payment_system_median        377346 non-null  float64
 10  ecommerce_flag_mean          377346 non-null  float64
 11  days_before_min              377346 non-null  int64  
 12  days_before_mean             377346 non-null  float64
 13 

In [5]:
x_train, x_test, y_train, y_test = train_test_split(my_xtrain, my_ytrain, test_size=0.1)

In [6]:
dataset = cb.Pool(x_train, y_train['flag'])

In [49]:
model1 = cb.CatBoostRegressor(loss_function="RMSE")
grid = {'iterations': [192, 195],
        'learning_rate': [ 0.1, 0.09],
        'depth': [  6, 7],
        'l2_leaf_reg': [3]}
model1.grid_search(grid, dataset)

0:	learn: 0.1628541	test: 0.1642264	best: 0.1642264 (0)	total: 18.5ms	remaining: 3.54s
1:	learn: 0.1623630	test: 0.1637424	best: 0.1637424 (1)	total: 33.9ms	remaining: 3.22s
2:	learn: 0.1619547	test: 0.1633258	best: 0.1633258 (2)	total: 47.7ms	remaining: 3s
3:	learn: 0.1616463	test: 0.1630218	best: 0.1630218 (3)	total: 66.1ms	remaining: 3.11s
4:	learn: 0.1613774	test: 0.1627568	best: 0.1627568 (4)	total: 82.5ms	remaining: 3.08s
5:	learn: 0.1611565	test: 0.1625350	best: 0.1625350 (5)	total: 102ms	remaining: 3.16s
6:	learn: 0.1609812	test: 0.1623715	best: 0.1623715 (6)	total: 116ms	remaining: 3.06s
7:	learn: 0.1608056	test: 0.1622087	best: 0.1622087 (7)	total: 134ms	remaining: 3.08s
8:	learn: 0.1606607	test: 0.1620685	best: 0.1620685 (8)	total: 150ms	remaining: 3.04s
9:	learn: 0.1605384	test: 0.1619553	best: 0.1619553 (9)	total: 171ms	remaining: 3.1s
10:	learn: 0.1604476	test: 0.1618700	best: 0.1618700 (10)	total: 189ms	remaining: 3.11s
11:	learn: 0.1603423	test: 0.1617752	best: 0.161775

{'params': {'depth': 7,
  'l2_leaf_reg': 3,
  'iterations': 195,
  'learning_rate': 0.09},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45

In [55]:
pred = model1.predict(x_test)
rmse = (np.sqrt(mean_squared_error(y_test['flag'], pred)))
r2 = r2_score(y_test['flag'], pred)
score = model1.score(x_test, y_test['flag'])
local_score = model1.score(x_train, y_train['flag'])
print("Testing performance")
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))
print("Score: {:.4f}".format(score))
print("Local Score: {:.4f}".format(local_score))
print("Best params: ", model1._get_params())

Testing performance
RMSE: 0.16
R2: 0.02
Score: 0.0208
Local Score: 0.0430
Best params:  {'depth': 7, 'verbose': 0, 'iterations': 195, 'l2_leaf_reg': 3, 'learning_rate': 0.09, 'loss_function': 'RMSE'}


### Get contest results

In [51]:
contest_x = pd.read_csv("../ourfeatures/updated_xtest.csv")
contest_y = pd.read_csv("../ourfeatures/my_big_testy.csv")

In [52]:
prediction = model1.predict(contest_x)
contest_y["flag"] = prediction

In [53]:
contest_y

,app_id,flag
0,805133,0.021097
1,805134,0.017119
2,805135,0.028804
3,805136,0.024360
4,805137,0.044933
...,...,...
188669,1003045,0.032815
188670,1003047,0.002272
188671,1003048,0.011812
188672,1003049,0.027930


In [54]:
contest_y.to_csv("results/result5.csv", index=False)